In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
#
# ASE is a very convenient module for setting up simulations on molecules and 
# bulk materials
#
from ase.io import read, write
from ase import Atoms
from ase.build import mx2, molecule
from ase.constraints import FixAtoms
from ase.visualize import view
from ase.calculators.espresso import Espresso, EspressoProfile
import time

In [42]:
MoS2 = mx2(formula='MoS2', kind='2H', a=3.18, thickness=3.19, size=(2, 2, 1), vacuum=10.)

In [43]:
MoS2.positions

array([[ 0.        ,  0.        , 11.595     ],
       [ 1.59      ,  0.91798693, 13.19      ],
       [ 1.59      ,  0.91798693, 10.        ],
       [-1.59      ,  2.75396078, 11.595     ],
       [ 0.        ,  3.67194771, 13.19      ],
       [ 0.        ,  3.67194771, 10.        ],
       [ 3.18      ,  0.        , 11.595     ],
       [ 4.77      ,  0.91798693, 13.19      ],
       [ 4.77      ,  0.91798693, 10.        ],
       [ 1.59      ,  2.75396078, 11.595     ],
       [ 3.18      ,  3.67194771, 13.19      ],
       [ 3.18      ,  3.67194771, 10.        ]])

In [44]:
# Optionally create profile to override paths in ASE configuration:
profile = EspressoProfile(
    command='mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x', pseudo_dir='./pseudos/'
)

In [45]:
pseudopotentials = {
    "H":"H.pbe-rrkjus_psl.1.0.0.UPF",
    "O":"O.pbe-n-kjpaw_psl.0.1.UPF",
    "Mo":"Mo_ONCV_PBE-1.0.oncvpsp.upf",
    "S":"s_pbe_v1.4.uspp.F.UPF"
}

In [118]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
    'ions':{
        },
    'cell': {
        'cell_dofree' : 'ibrav+2Dxy'
    }
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2.calc = calc

In [105]:
eMoS2 = MoS2.get_potential_energy()

CalledProcessError: Command '['mpirun', '-np', '4', '/Users/oliviero/PWSCF/espresso-git/bin/pw.x', '-in', 'espresso.pwi']' returned non-zero exit status 1.

In [106]:
MoS2 = read('MoS2_vcrelax.out')

In [107]:
MoS2.center()

In [108]:
MoS2.positions

array([[ 8.00261267e-04,  9.14688188e-01,  6.35012543e+00],
       [ 1.58511123e+00,  1.82998009e+00,  7.94349870e+00],
       [ 1.58511128e+00,  1.82997906e+00,  4.75675372e+00],
       [-1.58573460e+00,  3.66043743e+00,  6.35012558e+00],
       [-2.00093063e-03,  4.57641766e+00,  7.94411429e+00],
       [-2.00114573e-03,  4.57641758e+00,  4.75613867e+00],
       [ 3.17064002e+00,  9.15897827e-01,  6.35012497e+00],
       [ 4.75648140e+00,  1.82987738e+00,  7.94453638e+00],
       [ 4.75648244e+00,  1.82987791e+00,  4.75571733e+00],
       [ 1.58410661e+00,  3.66164705e+00,  6.35012469e+00],
       [ 3.16937385e+00,  4.57630852e+00,  7.94515188e+00],
       [ 3.16937409e+00,  4.57630853e+00,  4.75510112e+00]])

In [109]:
atop_index = 1
fixed = list(range(len(MoS2)))
fixed.remove(atop_index)
print(fixed)

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [110]:
constraint = FixAtoms(indices=fixed)
MoS2.set_constraint(constraint)

In [111]:
# Creating the OH molecule
oh_molecule = Atoms('OH', positions=[(0, 0, 0), (0, -0.763, 0.596)])

oh_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.5))

MoS2OH = MoS2 + oh_molecule

In [112]:
MoS2.positions[atop_index]

array([1.58511123, 1.82998009, 7.9434987 ])

In [113]:
oh_molecule.positions

array([[ 1.58511123,  1.82998009,  9.4434987 ],
       [ 1.58511123,  1.06698009, 10.0394987 ]])

In [114]:
MoS2OH.positions

array([[ 8.00261267e-04,  9.14688188e-01,  6.35012543e+00],
       [ 1.58511123e+00,  1.82998009e+00,  7.94349870e+00],
       [ 1.58511128e+00,  1.82997906e+00,  4.75675372e+00],
       [-1.58573460e+00,  3.66043743e+00,  6.35012558e+00],
       [-2.00093063e-03,  4.57641766e+00,  7.94411429e+00],
       [-2.00114573e-03,  4.57641758e+00,  4.75613867e+00],
       [ 3.17064002e+00,  9.15897827e-01,  6.35012497e+00],
       [ 4.75648140e+00,  1.82987738e+00,  7.94453638e+00],
       [ 4.75648244e+00,  1.82987791e+00,  4.75571733e+00],
       [ 1.58410661e+00,  3.66164705e+00,  6.35012469e+00],
       [ 3.16937385e+00,  4.57630852e+00,  7.94515188e+00],
       [ 3.16937409e+00,  4.57630853e+00,  4.75510112e+00],
       [ 1.58511123e+00,  1.82998009e+00,  9.44349870e+00],
       [ 1.58511123e+00,  1.06698009e+00,  1.00394987e+01]])

In [119]:
MoS2OH.calc = calc

In [120]:
from ase.optimize import QuasiNewton

In [121]:
dyn = QuasiNewton(MoS2OH, trajectory='MoS2OH.traj')
t = time.time()
dyn.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

CalledProcessError: Command '['mpirun', '-np', '4', '/Users/oliviero/PWSCF/espresso-git/bin/pw.x', '-in', 'espresso.pwi']' returned non-zero exit status 1.

In [ ]:
e_slab = slab.get_potential_energy()
print("Ni(111) energy: ", e_slab, " eV")

In [ ]:
# Creating the OH molecule and a new Ni slab
oh_molecule = Atoms('OH', positions=[(0, 0, 0), (0, -0.763, 0.596)])
slabNi = fcc111("Ni", a=lattice_constant_Ni, size=(2,2,2))

# Placing the molecule close to a top Ni atom, where it would bind
p = slabNi.positions[4]
oh_molecule.translate(p + (0, 0, 1.5))
slabOH = slabNi + oh_molecule
slabOH.center(vacuum=2.0)

In [ ]:
# We fix again some atoms of the slab to speed up calculations
constraint = FixAtoms(indices=[0,1,2,3,5,6,7])
slabOH.set_constraint(constraint)
view(slabOH)

In [ ]:
slabOH.set_calculator(calculator)

dynOH = QuasiNewton(slabOH, trajectory="OH_Ni.traj", )
t = time.time()
dynOH.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

e_slabOH = slabOH.get_potential_energy()
print("OH on Ni(111) energy: ", e_slabOH, " eV")
print("bond Ni-O: ", slabOH.get_distance(4,8))
print("bond O-H: ", slabOH.get_distance(8,9))

In [ ]:

"""
Calculations to obtain the free energy of the OH adsorbed on the Ni slab
"""



"""
Calculations to get the free energy of the O intermediate adsorbed on the Ni surface
"""
o_molecule = Atoms('O', positions=[(0, 0, 0)])
slabNi = fcc111("Ni", a=lattice_constant_Ni, size=(2,2,2))
p = slabNi.positions[4]
o_molecule.translate(p + (0, 0, 1.5))
slabO = slabNi + o_molecule
slabO.center(vacuum=2.0)
constraint = FixAtoms(indices=[0,1,2,3,5,6,7])
slabO.set_constraint(constraint)
view(slabO)

slabO.set_calculator(calculator)
dynO = QuasiNewton(slabO, trajectory="O_Ni.traj", )
t = time.time()
dynO.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

e_slabO = slabO.get_potential_energy()
print("O on Ni(111) energy: ", e_slabO, " eV")
view(slabO)
print("bond Ni-O: ", slabO.get_distance(4,8))


"""
Calculations to get the free energy of the OOH intermediate adsorbed on the Ni surface
"""
ooh_molecule = Atoms('OOH', positions=[(0, 0, 0), (0, 0, 1.4), (0, -0.763, 2.0)])
slabNi = fcc111("Ni", a=lattice_constant_Ni, size=(2,2,2))
p = slabNi.positions[4]
ooh_molecule.translate(p + (0, 0, 1.5))
slabOOH = slabNi + ooh_molecule
slabOOH.center(vacuum=2.0)
constraint = FixAtoms(indices=[0,1,2,3,5,6,7])
slabOOH.set_constraint(constraint)
view(slabOOH)

slabOOH.set_calculator(calculator)
dynOOH = QuasiNewton(slabOOH, trajectory="OOH_Ni.traj", )
t = time.time()
dynOOH.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

e_slabO = slabOOH.get_potential_energy()
print("OOH on Ni(111) energy: ", e_slabO, " eV")
view(slabOOH)
print("bond Ni-O: ", slabOOH.get_distance(4,8))




"""
Calculations to get the free energy of H2O and H2 needed to calculate the free energies of the reaction
"""

calculator2 = GPAW(xc='RPBE', mode=PW(350), h=0.2, occupations=FermiDirac(0.1))
h2o_molecule = molecule("H2O")
h2o_molecule.set_cell(slab.get_cell())
h2o_molecule.center(vacuum=2.0)
h2o_molecule.set_calculator(calculator2)
dynH2O = QuasiNewton(h2o_molecule, trajectory='H2O.traj')
t = time.time()
dynH2O.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))
e_h2o = h2o_molecule.get_potential_energy()
print("H2O energy: ", e_h2o, " eV")
print("O-H bond lenght: ", h2o_molecule.get_distance(0,1))


h2_molecule = molecule("H2")
h2_molecule.set_cell(slab.get_cell())
h2_molecule.center(vacuum=2.0)
h2_molecule.set_calculator(calculator2)
dynH2 = QuasiNewton(h2_molecule, trajectory='H2.traj')
t = time.time()
dynH2.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))
e_h2 = h2_molecule.get_potential_energy()
print("H2 energy: ", e_h2, " eV")
print("H-H bond lenght: ", h2_molecule.get_distance(0,1))